In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
pip install -U 'scikit-learn<0.24'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 4.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [3]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.6 MB/s 


In [4]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from nltk.tokenize import word_tokenize

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
from google.colab import drive,files
drive.mount("/content/drive")
dataM='/content/drive/My Drive/Colab Notebooks/MODELS/'
filepath = dataM+'CRF.ml'

Mounted at /content/drive


In [7]:
!cp -rf drive/MyDrive/job_20k.json /content/

In [8]:
import pandas as pd
import json
path_to_json = 'job_20k.json'
df  = pd.read_json(path_to_json,lines=True)
df.head().T

,0,1,2,3,4
clean_descrition,A technology services client of ours is lookin...,Job Title :Java Sr developerLocation: RemoteLo...,Mandatory Skills: Strong in Oracle database 19...,"Associate Scientist Thousand Oaks, CADescripti...","Growing fintech company looking for Mid Level,..."
organization_list,"[Swift, Strategic, Solutions, Inc]","[AQUA, Information, Systems,, Inc.]","[Softnet, Consulting, Inc]","[Apex, Systems]",[Jobot]
skill_list,"[NoSQL, Teradata, Netezza, Cosmos, DB, JSON, A...","[Java, JEE, J2EE]","[Oracle, database, 19c, Advanced, PL/SQL, prog...","[Biochemistry, Biology, Analytical, Systems]","[Developer, JavaScript, SQL, ASP, HTML, Archit..."
job_title_list,"[Bigdata, Engineer]","[Java, developer]","[Oracle, PL/SQL, Lead, Architect]","[Associate, Scientist]","[Software, Engineer]"


In [9]:
df['clean_descrition'] = df['clean_descrition'].str.replace(r'[^\w\s]+', ' ')
df['clean_descrition'] = df['clean_descrition'].apply(lambda x: ' '.join(x.split()[:128]))
df['clean_descrition'][1]

<ipython-input-9-42c4f31900d0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_descrition'] = df['clean_descrition'].str.replace(r'[^\w\s]+', ' ')


'Job Title Java Sr developerLocation RemoteLong TermSkills java jeShould have good understanding on the change management role and had depth knowledge on the change advisory board process CAB Good knowledge on the incident management Problem management Change Management Should have involved in the 3L level of activities i e root cause analysis environment analysis understanding the application architecture infrastructure architecture The resource should have very good understanding on the SDLC process Very good knowledge on the Java J2EE and have hands on experience in understanding the architecture of heterogeneous application Good communication skills and team handling abilities Should have worked on the onshore offshore model Should be flexible and having good attitude'

In [10]:
jobs = list(df.clean_descrition)
def convert_to_list_then_dict(df_column= None, key_value=str):
    value_list = list(df_column)
    _ = list(set([item for value_list in value_list for item in value_list]))
    return dict.fromkeys(_, key_value)

org = convert_to_list_then_dict(df_column=df.organization_list, key_value='C')
skill = convert_to_list_then_dict(df_column=df.skill_list, key_value='S')
job = convert_to_list_then_dict(df_column=df.job_title_list, key_value='J')    

In [11]:
XTr= jobs[:1500]
XVal= jobs[1500:2000]
org_b = org
skills_b = skill
jobs_b = job

In [12]:
def make(X,org_b,skills_b,jobs_b):
  big=[]
  for sentence in X:
    token=word_tokenize(sentence)
    small=[]
    token_t=nltk.pos_tag(token)
    for f,s in token_t:
      tag='O'
      if f.lower() in org_b:
        tag=org_b[f.lower()]
      if f.lower() in skills_b:
        tag=skills_b[f.lower()]
      if f.lower() in jobs_b:
        tag=jobs_b[f.lower()]
      small.append((f,s,tag))
    big.append(small)
  return big
XTr_=make(XTr,org_b,skills_b,jobs_b)
XVal_=make(XVal,org_b,skills_b,jobs_b)

In [13]:
def getfeatures(dat,i):
  word=dat[i][0]
  postag=dat[i][1]
  features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
  }
  if i>0:
    word=dat[i-1][0]
    postag=dat[i-1][1]
    features.update({
        '-1:word.lower()': word.lower(),    
        '-1:word.isupper()': word.isupper(),
        '-1:word.istitle()': word.istitle(),
        '-1:postag': postag,
        '-1:postag[:2]': postag[:2]
    })
  else:
    features['BOS']=True
  if i+1<len(dat):
    word=dat[i+1][0]
    postag=dat[i+1][1]
    features.update({
        '+1:word.lower()': word.lower(),    
        '+1:word.isupper()': word.isupper(),
        '+1:word.istitle()': word.istitle(),
        '+1:postag': postag,
        '+1:postag[:2]': postag[:2]
    })
  else:
    features['EOS']=True
  return features

In [14]:
def f2features(dat):
    return [getfeatures(dat, i) for i in range(len(dat))]

def f2labels(dat):
    return [label for token, postag, label in dat]

def f2tokens(dat):
    return [token for token, postag, label in dat]

In [15]:
XTr__=[f2features(x) for x in XTr_]
YTr__=[f2labels(x) for x in XTr_]

XVal__=[f2features(x) for x in XVal_]
YVal__=[f2labels(x) for x in XVal_]

In [16]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [17]:
crf.fit(XTr__, YTr__)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [22]:
labls=crf.classes_

In [23]:
y_pred = crf.predict(XVal__)
metrics.flat_f1_score(YVal__, y_pred,average='weighted', labels=labls)

0.9937340122500872

In [24]:
print(metrics.flat_classification_report(YVal__, y_pred,digits=3))

              precision    recall  f1-score   support

           C      1.000     0.953     0.976        43
           J      0.999     0.999     0.999     32559
           O      0.983     0.986     0.985     11927
           S      0.991     0.988     0.990     15148

    accuracy                          0.994     59677
   macro avg      0.993     0.982     0.987     59677
weighted avg      0.994     0.994     0.994     59677



In [26]:
print(y_pred[:10])

[['O', 'J', 'J', 'O', 'J', 'O', 'J', 'J', 'J', 'O', 'J', 'S', 'O', 'O', 'O', 'O', 'J', 'O', 'S', 'S', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'O', 'J', 'J', 'J', 'O', 'O', 'J', 'S', 'O', 'O', 'S', 'S', 'J', 'J', 'S', 'J', 'S', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'O', 'J', 'O', 'S', 'J', 'S', 'S', 'J', 'S', 'J', 'O', 'S', 'J', 'J', 'J', 'S', 'O', 'O', 'O', 'J', 'S', 'J', 'S', 'O', 'O', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'S', 'J', 'J', 'O', 'J', 'S', 'J', 'O', 'J', 'J', 'J', 'O', 'J', 'O', 'O', 'J', 'J', 'O', 'S', 'J', 'S', 'J', 'O', 'S', 'S', 'J', 'S', 'J', 'J', 'S', 'J', 'O', 'J', 'O', 'J', 'S', 'S', 'S', 'O', 'J', 'J', 'S'], ['J', 'O', 'J', 'J', 'J', 'J', 'O', 'S', 'J', 'J', 'S', 'S', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'J', 'S', 'J', 'J', 'S', 'O', 'J', 'J', 'O', 'J', 'S', 'J', 'O', 'J', 'S', 'S', 'O', 'J', 'J', 'J', 'O', 'J', 'J', 'J', 'O', 'J', 'J', 'J', 'O', 'J', 'O', 'J', 'S', 'J', 'J', 'S', 'S', 'S', 'J', 'J', 'J', 'J', 'S', 'J', 'J', 'J', 'J', 'J', 'O', 'J', 'J', 'J', '